# Logistic Regression

## Numerical Optimization
The Logistic Regression model is obtained by **minimizing the average cross-entropy between the model predictions and the observed labels**. As we have seen, this corresponds also to a **Maximum Likelihood solution for the observed labels**, or to **minimizing the average Logistic Loss function.** See the next section for all the theory related to Logistic Regression, here we just report the final formulas for the three objective functions:
- log-likelihood: $l(\mathbf{w}, b) = \sum_{i=1}^{n} \left[c_i \log y_i + (1 - c_i) \log(1 - y_i)\right] \rightarrow$ GOAL: maximize $l(\mathbf{w}, b)$ wrt $\mathbf{w}, b$.

- average cross-entroy: $\mathcal{J}(\mathbf{w}, b) = -l(\mathbf{w}, b) = - \sum_{i=1}^{n} \left[c_i \log y_i + (1 - c_i) \log(1 - y_i)\right] \rightarrow$ GOAL: minimize $\mathcal{J}(\mathbf{w}, b)$ wrt $\mathbf{w}, b$.

- average Logistic Loss function: $\mathcal{J}(\mathbf{w}, b) = \sum_{i=1}^{n} \log(1 + e^{-z_i(\mathbf{w}^T \mathbf{x}_i + b)})  \rightarrow$ GOAL: minimize $\mathcal{J}(\mathbf{w}, b)$ wrt $\mathbf{w}, b$.

While for Gaussian models closed form expressions are available for the
ML solutions, this is not the case for Logistic Regression. This means, **we can't just solve system of equations to find the optimal parameters**. This is because the **sigmoid** function involved in binary Logistic regression (and the **softmax** function involved in multiclass Logistic regression) make the loss function nonlinear and non-convex in general<br>
Therefore, we turn to numerical optimization
to find the maximizer of the class likelihoods, or, equivalently, the minimizer of the average cross-entropy or average Logistic Loss function. <br>
Numerical optimization algorithms look for the minimum of a function $f(x)$ with respect to the argument
$x$. Here we briefly explain two methods, the second one will be the one adopted by us:
### 1) Gradient Descent (GD)
with this iterative method, at each iteration $t$ we compute $x_{t+1}$ from $x_{t}$:
- we compute the gradient $\nabla f(x_t)$ of the loss function with respect to the current parameters $x_t$.
- we then update the parameters by moving in the **opposite direction of the gradient** (this is done by multiplying the gradient by $-1$), scaled by a learning rate (also called step) $\alpha_t$:

$$
x_{t+1} = x_t - \alpha_t \nabla f(x_t)
$$

Under the assumptions that the step becomes lower when iterations pass $\left( \alpha_t \rightarrow 0 \right)$ and that the whole sum of all the steps at each iteration is unbounded $\left( \sum_{t=1}^{\infty} \alpha_t \rightarrow \infty \right)$, we are certain that the algorithm converges to a **local minimum** of $f$.
#### Pros of GD
- Easy to implement  
- Low memory usage

#### Cons of GD
- Can be **very slow to converge**  
- Sensitive to choice of learning rate  
- Struggles with ill-conditioned loss surfaces

### 2) L-BFGS (Limited-memory Broyden–Fletcher–Goldfarb–Shanno)
L-BFGS is a more advanced optimization algorithm that uses an **approximate second-order method**. Instead of relying solely on the gradient, it also uses curvature information so second order information, such as the Hessian of the function, from previous iterations to guide the search more efficiently. <br>

#### Pros of L-BFGS
- Much **faster convergence** than gradient descent  
- No need to compute or store the full Hessian (which would be $\mathcal{O}(d^2)$)

#### Cons of L-BFGS
- Slightly more complex and higher per-iteration cost ($\mathcal{O}(md)$, whereas GD has just $\mathcal{O}(d)$)


This second algorithm is the one we''l use and is implemented in `scipy` (requires importing `scipy.optimize`). We will use the `scipy.optimize.fmin_l_bfgs_b` interface to the numerical solver.

`scipy.optimize.fmin_l_bfgs_b` requires at least 2 arguments (check the documentation for more details):

* `func`: the function we want to minimize.
* `x0`: the starting value for the algorithm.

The L-BFGS algorithm requires computing the objective function and its gradient. To pass the gradient we have different options:

* Through `func`: `func` should return a tuple `(f(x), \nabla_x f(x))`.
* Through the optional parameter `fprime`: `fprime` is a function computing the gradient. In this case, `func` should only return the objective value $f(x)$.
* Let the implementation compute an approximated gradient: pass `approx_grad = True`. In this case, `func` should only return the objective value $f(x)$.

The last option does not require writing a function that computes the gradient, as an approximation of the gradient is automatically obtained through finite differences. While this has the advantage that we do not need to derive and implement the gradient, it has two drawbacks:

* The gradient computed through finite differences may not be accurate enough.
* The computations are much more expensive, since we need to evaluate the objective function a number of times at least $D$, where $D$ is the size of $x$, at each iteration, and if we want a more accurate approximation of the gradient we may need to evaluate $f$ many more times.



As an example, we now try to apply the L-BFGS to the function:
$$
f(y, z) = (y + 3)^2 + \sin(y) + (z + 1)^2
$$

In [1]:
import numpy as np
import scipy.optimize as opt

In [2]:
#implementation of function f(y,z)
def f(x):
    #x is an numpy array of shape (2,)
    #x[0] is y and x[1] is z
    #te function returns the value of f(y,z) = (y+3)^2 + sin(y) + (z+1)^2
    y = x[0]
    z = x[1]

    return (y+3)**2 + np.sin(y) + (z+1)**2


#Now we call scipy.optimize.fmin_l_bfgs_b passing the function f and the initial x0 which is a numpy array of values [0,0] and approx_grad = True
x_0 = np.array([0, 0])

#x_min is the minimum point of the function f
#f_min is the value of the function f at the minimum point x_min
#d is a dictionary with information about the optimization process
x_min, f_min, d = opt.fmin_l_bfgs_b(f, x_0, approx_grad=True)

print(f"Mimimum of f(y,z) is at x_min = {x_min}")
print(f"f(y_min,z_min) = f(x_min) = {f_min}")
print(f"Optimization info: {d}")

Mimimum of f(y,z) is at x_min = [-2.57747138 -0.99999927]
f(y_min,z_min) = f(x_min) = -0.3561430123647649
Optimization info: {'grad': array([-1.49324998e-06,  1.46549439e-06]), 'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 'funcalls': 21, 'nit': 6, 'warnflag': 0}


We can check the number of times the function $f$ was called with this second approach:

In [3]:
print(f"function f called {d['funcalls']} times")

function f called 21 times


We can also provide an explicit gradient, in this case te function is very simple so rather than approximate it we can just compute the two partial derivatives explicitly:

In [4]:
def f1(x):
    #x is an numpy array of shape (2,)
    #x[0] is y and x[1] is z
    #te function returns the value of f(y,z) = (y+3)^2 + sin(y) + (z+1)^2
    y = x[0]
    z = x[1]

    #provide explicit gradient of f
    y_derivative = 2*(y+3) + np.cos(y)
    z_derivative = 2*(z+1)

    #gradient has shape (2,)
    return (y+3)**2 + np.sin(y) + (z+1)**2, np.array([y_derivative, z_derivative])

#Now we call scipy.optimize.fmin_l_bfgs_b passing the function f and the initial x0 which is a numpy array of values [0,0] and pass the explicitly computed  gradient of f
x_0 = np.array([0, 0])

#x_min is the minimum point of the function f
#f_min is the value of the function f at the minimum point x_min
#d is a dictionary with information about the optimization process
x_min, f_min, d = opt.fmin_l_bfgs_b(f1, x_0)

print(f"Mimimum of f(y,z) is at x_min = {x_min}")
print(f"f(y_min,z_min) = f(x_min) = {f_min}")
print(f"Optimization info: {d}")

Mimimum of f(y,z) is at x_min = [-2.57747137 -0.99999927]
f(y_min,z_min) = f(x_min) = -0.3561430123647611
Optimization info: {'grad': array([-1.50318729e-06,  1.46120529e-06]), 'task': 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL', 'funcalls': 7, 'nit': 6, 'warnflag': 0}


We can check the number of times the function $f$ was called with this second approach:

In [5]:
print(f"function f called {d['funcalls']} times")

function f called 7 times


Whereas with the first approach (i.e. let scipy automatically approximate the gradient) $f$ was called 21 times, in this second case it's called just 7 times, a third! <br>
So, we can say that te automatic numerical approximation of the gradient is significantly more expensive, and the cost becomes
relatively worse when the dimensionality of the domain of f increases.

## Binary logistic regression
It's important to underline that, althougth it's in the name, this is a classification model (like the GGM), and not a regression model (such as linear regression). <br>
We introduce this other model because in this case our goal becomes very different from when we studied the GGM. If, when we implemented and applied the GGM, our goal was to model the distribution of the observed samples $X \mid C$, when using Logistic Regression we want to directly model the class posteriors distribution $C \mid X$. <br>
Using again a generative approach, the Posterior probability for class $h_1$ can be computed from the modeled priors and the modeled class conditional densities, using the Bayes' Theorem:
$$
P(C = h_1 \mid \mathbf{x}) = \frac{f_{\mathbf{X} \mid C}(\mathbf{x} \mid C = h_1) P(C = h_1)}{\sum_{i=0}^{K-1} f_{\mathbf{X} \mid C}(\mathbf{x} | C = c_i) P(C = c_i)} = \frac{f_{\mathbf{X} \mid C}(\mathbf{x} \mid C = h_1) P(C = h_1)}{ f_{\mathbf{X} \mid C}(\mathbf{x} | C = h_0) P(C = h_0) + f_{\mathbf{X} \mid C}(\mathbf{x} | C = h_1) P(C = h_1)}
$$
This can be rewritten as:
$$
P(C = h_1 \mid \mathbf{x}) = \frac{1}{1+ e^{-s(\mathbf{x})}} = \sigma(s(\mathbf{x}))
$$
Where:
- $s(\mathbf{x})$ is the score

- $\sigma (t) = \frac{1}{1+ e^{-t}}$ is called **sigmoid**/logistic function. It has $\lim_{t \to - \infty} \sigma(t) = 0$ and $\lim_{t \to \infty} \sigma(t) = 1$. An important property of this function is that: $1 - \sigma (t) = \sigma (-t)$ <br>
<img src="sigmoid.png" alt="image.png" style="background-color: #ADD8E6; width: 400px; height: 200px;">

For the score, we can use the **log-posterior ratio**, given in the log domain by the sum of the log-likelihood ratio and the prior log odds, as we already did with the GGM:
$$
s(\mathbf{x}) = llr(\mathbf{x}) + \text{log}\frac{\pi}{1+\pi} = \text{log} \frac{f_{\mathbf{X} \mid C}(\mathbf{x} | C = h_1)}{f_{\mathbf{X} \mid C}(\mathbf{x} | C = h_0)} + \text{log}\frac{\pi}{1+\pi}
$$
In this first phase, we can impose and use **linear classification rules**, so the score can be rewritten as:
$$
s(\mathbf{x}) = \mathbf{w}^T \mathbf{x} + b
$$
This expression is linear wrt features $\mathbf{x}$. $\mathbf{w}$ is the weigth vector and is **orthogonal** to the linear decision surface, $b$ is a scalar bias, which also absorbes information from the Priors. $s(\mathbf{x})$ is positive for samples of class $h_1$ and negative for samples of class $h_0$. The equation:
$$
s(\mathbf{x}) = \mathbf{w}^T \mathbf{x} + b = 0
$$
defines the linear decision surface, so the linear hyperplane, which separates our classes. <br>
So, to sum up all of this, we can write:
$$
P(C = h_1 \mid \mathbf{x}, \mathbf{w}, b) = \sigma(\mathbf{w}^T \mathbf{x} + b)
$$
Since we're speaking about probabilities and we just have two classes, we can also write the posterior for the other class, $h_0$, as:
$$
P(C = h_0 \mid \mathbf{x}, \mathbf{w}, b) = 1 - P(C = h_1 \mid \mathbf{x}, \mathbf{w}, b) = 1 - \sigma(\mathbf{w}^T \mathbf{x} + b) = \sigma(- \mathbf{w}^T \mathbf{x} + b)
$$
So, it's clear that we cannot compute these posteriors without knowing the model parameters, $(\mathbf{w}, b)$. <br>
Now, assuming we have a labeled training dataset $\mathcal{D} = [(\mathbf{x}_1, c_1), \ldots, (\mathbf{x}_n, c_n)]$ where classes are independently distributed, we can express the likelihood for the observed labels as
$$
\mathcal{L}(\mathbf{w}, b) = P(C_1 = c_1, \ldots, C_n = c_n | \mathbf{x}_1, \ldots, \mathbf{x}_n, \mathbf{w}, b) = \prod_{i=1}^{n} P(C_i = c_i | \mathbf{x}_i, \mathbf{w}, b)
$$

We can thus apply a **ML approach** to estimate the model parameters that best describe the observed labels $(c_1, \ldots, c_n)$. So we want to find the value of $\mathbf{w}$ and b that maximize the likelihood of our training labels. <br>
We assume that the classes $h_1, h_0$ have labels 1 and 0 respectively. Also, let $y_i = P(C_i = 1 | \mathbf{x}_i, \mathbf{w}, b) = \sigma(\mathbf{w}^T \mathbf{x}_i + b)$. It follows that $P(C_i = 0 | \mathbf{x}_i, \mathbf{w}, b) = 1 - y_i$. <br>
So, the distribution for $C_i | \mathbf{x}_i, \mathbf{w}, b$ is a Bernoulli distribution:

$$
C_i | \mathbf{x}_i, \mathbf{w}, b \sim \text{Ber}(\sigma(\mathbf{w}^T \mathbf{x}_i + b)) = \text{Ber}(y_i)
$$

We can thus rewrite the likelihood using the Bernoulli formula for the density:

$$
\mathcal{L}(\mathbf{w}, b) = \prod_{i=1}^{n} y_i^{c_i} (1 - y_i)^{(1 - c_i)}
$$


As always working with the log-likelihood is more practical, so we have

$$
l(\mathbf{w}, b) = \sum_{i=1}^{n} [c_i \log y_i + (1 - c_i) \log(1 - y_i)] = \sum_{i=1}^{n} [c_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) \ + (1 - c_i) \log \left(1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b) \right)] \tag{1 way}
$$
This is the first way to write the L.R. Objective function. There are other three ways. All of this formulas have a tag on the right which tells you they are all the ways to write the objective function. <br>
Our goal is the maximization of $l$ with respect to $\mathbf{w}$ and $b$.
As briefly said before, we can do this but also follow other two approaches involving the minimization of the average cross-entropy or the minimization of the average Logistic Loss function. <br>
The Average cross-entropy measures how good are the predictions made by te Recognizer, versus the actual labels present in the training data and it's just the negative of the likelihood:
$$
\mathcal{J}(\mathbf{w}, b) = - \sum_{i=1}^{n} [c_i \log y_i + (1 - c_i) \log(1 - y_i)] = \sum_{i=1}^{n} - [c_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) \ + (1 - c_i) \log \left(1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b) \right)] \tag{2 way}
$$
Some numerical issues that may arise when explicitly computing sigmoids followed by natural logarithm, so we can rewrite the formula for the average cross entropy by excplicitly plugging in the sigmoid formula and observing that:
$$
\log \sigma(\mathbf{w}^T \mathbf{x}_i + b) = -\log (1 + e^{-\mathbf{w}^T \mathbf{x}_i - b}) \\ \\
\log (1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b)) = -\log \sigma(-\mathbf{w}^T \mathbf{x}_i - b) = -\log (1 + e^{\mathbf{w}^T \mathbf{x}_i + b})
$$
Thus we can also write the average cross-entropy this way:
$$
\mathcal{J}(\mathbf{w}, b) = \frac{1}{n} \sum_{i=1}^{n} \left[c_i \log (1 + e^{-\mathbf{w}^T \mathbf{x}_i - b}) + (1 - c_i) \log (1 + e^{\mathbf{w}^T \mathbf{x}_i + b})\right] \tag{3 way}
$$

The Logistic Loss function is just another way to see all of this, although from an algebric standpoint it's just equivalent to the average cross-entropy after some semplifications and a substitution. As a matter of facts, let:
$$
z_i = 2c_i - 1 \implies
\begin{cases}
    z_i = 2* 0 - 1 = -1 & \text{if } c_i = 0 \\
    z_i = 2 * 1 - 1 = 1  & \text{if } c_i = 1
\end{cases}
$$
So, considering just a single sample $\mathbf{x}_i$, we can see that the binary cross entropy for predicted label $y_i$ and an actual label $c_i$ is:
$$
\mathcal{H}(c_i, y_i) = - [c_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) \ + (1 - c_i) \log \left(1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b) \right)] =
\begin{cases}
    - \log \sigma (\mathbf{w}^T \mathbf{x}_i + b) & \text{if } c_i = 1 \iff z_i = 1 \\
    - \log \sigma (- \mathbf{w}^T \mathbf{x}_i + b) & \text{if } c_i = 0 \iff z_i = -1
\end{cases} =
- \log \sigma (z_i \cdot (\mathbf{w}^T \mathbf{x}_i + b))
$$
Thus we can rewrite the average cross entropy across all samples as:
$$
\mathcal{J}(\mathbf{w}, b) = \frac{1}{n} \sum_{i}^{n} \mathcal{H}(c_i, y_i) = \frac{1}{n} \sum_{i}^{n} \left[ - \log \sigma (z_i \cdot (\mathbf{w}^T \mathbf{x}_i + b)) \right]
$$
If we explicitly write the sigmoid formula and make some semplifications we get:

$$
\mathcal{J}(\mathbf{w}, b) = \frac{1}{n} \sum_{i}^{n} \log \left(  1 + e^{-z_i \cdot (\mathbf{w}^T \mathbf{x}_i + b)} \right) \tag{4 way}
$$
This is called **average Logistic Loss function**. It measures the empirical risk which is defined by the cost to pay due to missclassification of samples, or even due uncertainty despite the classification is correct.<br>

The last thing to talk about is **regularization**. In fact, if the classes are completely **separable** (i.e. we don't have samples from one class mixed with other from other classes), the L.R. solution is **undefined**, because we can aribtrarily decrease the Logistic Loss by arbitrarily increasing either $\|\mathbf{w}\|$ or b, so the score $\mathbf{w}^T \mathbf{x}_i + b$ can get extremely high. This means the optimal solution is obtained where $\|\mathbf{w}\| \to \infty$. In this case, to avoid having an undefined solution (and a potential freeze and/or crash of our program), we can manually impose a **norm penalty** to the objective function, so, when looking for the optimal solution, we'll have to respect this contraint as well and won't be able to increase the norm of $\mathbf{w}$ more and more towards $\infty$. 
The regularization term (norm penalty term) is composed like this:
$$
\frac{\lambda}{2} \|\mathbf{w}\|^2
$$
It's important to note that $\lambda$ is a **hyperparameter** of the model and its optimal value
may be found with **cross-validation**. In fact, $\lambda$ can’t be obtained by trying to
minimize the objective function with respect to $\lambda$ , as we would obtain $\lambda = 0$ which would remove
the regularization term from the equation. <br>
So, we can apply regularization and rewrite the three objective functions (for now we don't rewrite the log likelihood anymore because it's the only one which needs to be maximized, and in this case $\lambda$ would have to have the opposite sign wrt all the other cases..so we just write the two ways of expressing the average cross entropy, and the Logistic Loss function):
$$
\mathcal{J}(\mathbf{w}, b) = \frac{\lambda}{2} \|\mathbf{w}\|^2 - \frac{1}{n} \sum_{i=1}^{n} [c_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) + (1 - c_i) \log(1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b))] \quad (1)
$$

$$
\mathcal{J}(\mathbf{w}, b) = \frac{\lambda}{2} \|\mathbf{w}\|^2 + \frac{1}{n} \sum_{i=1}^{n} \log (1 + e^{-z_i(\mathbf{w}^T \mathbf{x}_i + b)}), \quad z_i = \begin{cases} 1 & \text{if } c_i = 1 \\ -1 & \text{if } c_i = 0 \end{cases} \quad (i.e. \ z_i = 2c_i - 1) \quad (2)
$$

$$
\mathcal{J}(\mathbf{w}, b) = \frac{\lambda}{2} \|\mathbf{w}\|^2 + \frac{1}{n} \sum_{i=1}^{n} [c_i \log (1 + e^{-\mathbf{w}^T \mathbf{x}_i - b}) + (1 - c_i) \log (1 + e^{\mathbf{w}^T \mathbf{x}_i + b})] \quad (3)
$$


**Let's implement the Logistic Regression using (2) as objective function.** <br>
First, let's import the dataset and apply the train-test split: <br>
We will represent labels with 1 (iris versicolor) and 0 (iris virginica).

In [11]:
import sklearn.datasets
from train_validation_split import splitTrainingValidation
from mean_covariance import vcol

def load_iris_binary():
    D, L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']
    D = D[:, L != 0] # We remove setosa from D
    L = L[L!=0] # We remove setosa from L
    L[L==2] = 0 # We assign label 0 to virginica (was label 2)
    return D, L


D, L = load_iris_binary()
(DTR, LTR), (DVAL, LVAL) = splitTrainingValidation(2/3, D, L)
print(f"Training set shape: {DTR.shape}")
print(f"Validation set shape: {DVAL.shape}")
print(f"Training set labels: {LTR}")
print(f"Validation set labels: {LVAL}")


Training set shape: (4, 66)
Validation set shape: (4, 34)
Training set labels: [1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1
 0 0 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 0 0 0 0]
Validation set labels: [1 0 1 0 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1]


- Function logreg_obj should receive a single numpy array v with shape (D+1,), where D is the
dimensionality of the feature space (e.g. D = 4 for IRIS). v should pack all model parameters, i.e.
v = [w, b]. Inside the function you can then unpack the array e.g. w, b = v[0:-1], v[-1].
- The function logreg_obj needs to access also DTR, LTR and $\lambda$, which are required to compute
the objective function. So, we embed the objective function and its optimization inside a function trainLogReg that receives both DTR, LTR and a value for l. The objective function is defined inside trainLogReg,
thus it can access the variables DTR and LTR in the outer function scope (i.e., those passed to
trainLogReg)
- The computation of $\log(1 + e^{-z_i(\mathbf{w}^T \mathbf{x}_i + b)})$ can lead to numerical issues when $z_i(\mathbf{w}^T \mathbf{x}_i + b)$ is large, since the sum will make the contribution of the exponential term disappear. We can avoid the issue by using the `numpy.logaddexp` function, which computes:
    $$
    \text{numpy.logaddexp}(a, b) = \log(e^a + e^b)
    $$
    In our example, we need to compute `numpy.logaddexp(0, -z_i(\mathbf{w}^T \mathbf{x}_i + b))`.
- Broadcasting can significantly speed-up the computations. You can compute a vector of “scores” $\mathbf{S}$

    $$
    \mathbf{S} = [(\mathbf{w}^T \mathbf{x}_1 + b) \ldots (\mathbf{w}^T \mathbf{x}_n + b)]
    $$

    using simple matrix-vector multiplication: `S = (vcol(w).T @ DTR + b).ravel()`. Remember to reshape the result to a 1-D array (`.ravel()`). You can then multiply each element of the result-vector by the corresponding label: `-ZTR * S`, where `ZTR = 2 * LTR - 1`. `numpy.logaddexp` supports broadcasting so you can compute in a single shot all terms

    $$
    \log \left(1 + e^{-z_i(\mathbf{w}^T \mathbf{x}_i + b)}\right)
    $$

    with `numpy.logaddexp(0, -ZTR * S)`.

- To speed-up computations, it's useful to provide also the gradient of the function. We can express in vector form the derivatives with respect to the components of $\mathbf{w}$ and $b$ as (notice that there is no minus in the exponential term $z_i(\mathbf{w}^T \mathbf{x}_i + b)$):

    $$
    \nabla_{\mathbf{w}} J = \left[\frac{\partial J}{\partial w_j}\right] = \lambda \mathbf{w} + \frac{1}{n} \sum_{i=1}^{n} \frac{-z_i}{1 + e^{z_i(\mathbf{w}^T \mathbf{x}_i + b)}} \mathbf{x}_i
    $$

    $$
    \frac{\partial J}{\partial b} = \frac{1}{n} \sum_{i=1}^{n} \frac{-z_i}{1 + e^{z_i(\mathbf{w}^T \mathbf{x}_i + b)}}
    $$

    where $d$ is the dimensionality of the feature vectors (`DTR.shape[0]`). These terms can be efficiently computed by computing the vector $\mathbf{G}$:

    $$
    \mathbf{G} = \left[\frac{-z_1}{1 + e^{z_1(\mathbf{w}^T \mathbf{x}_1 + b)}} \ldots \frac{-z_n}{1 + e^{z_n(\mathbf{w}^T \mathbf{x}_n + b)}}\right]
    $$

    so that

    $$
    \nabla_{\mathbf{w}} J = \lambda \mathbf{w} + \frac{1}{n} \sum_{i=1}^{n} G_i \mathbf{x}_i
    $$

    $$
    \frac{\partial J}{\partial b} = \frac{1}{n} \sum_{i=1}^{n} G_i
    $$

    which can be computed, using broadcasting, from the vector of scores $\mathbf{S}$ as `G = -ZTR / (1.0 + numpy.exp(ZTR * S))` (note: you may obtain overflow from the exponentiation due to well-classified samples, however you can safely ignore the issue since it will simply cause the corresponding $G_i$ becoming exactly 0 rather than almost 0, and this will not cause numerical issues). You can then compute all terms $\mathbf{G} \mathbf{x}_i$ with broadcasting: `(vrow(G) * DTR)`.

    Remember that `scipy.optimize.fmin_l_bfgs_b` requires that you pack $\nabla_{\mathbf{w}} J$ and $\frac{\partial J}{\partial b}$ in a single vector `vgrad = [$\nabla_{\mathbf{w}} J$, $\frac{\partial J}{\partial b}$]`. You have to modify the function `logreg_obj` so that it returns both the objective and the gradient `vgrad` (and set `approx_grad = False` when calling the optimizer function).

- $\lambda$ is a hyper-parameter. As usual, we should employ a validation set to estimate good values of $\lambda$. For this laboratory, we can simply try different values and see how this affects the performance.

- The starting point does not significantly influence the result, since the objective function is convex (there may be slight differences, but should be very small). You can use as initial value an array of all zeros `x0 = numpy.zeros(DTR.shape[0] + 1)`.

In [49]:
def trainLogReg(DTR, LTR, l): 
    """
    Train a logistic regression classifier using LTR as labels and DTR as data.
    l is the regularization parameter (lambda).
    """

    #LTR: ACTUAL labels -> they the c_i
    #compute z_i = 2 * c_i - 1 -> ZTR = 2 * LTR - 1
    #z_i is used in the Logistic Loss, not c_i
    ZTR = 2 * LTR - 1


    def logreg_obj(v):
        """
        Compute the Objective function for logistic regression.
        v is the vector of parameters in the form g. w, b = v[0:-1], v[-1]
        Parameter:
        - v: numpy array of shape (n_features + 1,)
        Returns:
        - f: float, the value of the objective function
        """
        #extract w and b from v
        w = v[:-1]  #weights
        b = v[-1]   #bias   

        #Now, the objective function is the Logistic Loss which is:
        #f(w, b) = 0.5 * l * ||w||^2 + 1/n * sum_i=1^n log(1 + exp(-z_i * (w^T x_i + b)))
        #so it's the sum of two terms:
        #1. the regularization term: 0.5 * l * ||w||^2
        #2. the average logistic loss term: 1/n * sum_i=1^n log(1 + exp(-z_i * (w^T x_i + b)))


        #compute regularization term (= norm penality)
        normPenalty = 0.5 * l * np.linalg.norm(w)**2

        #to compute the term: log (1 + exp(-z_i * (w^T x_i + b)))
        #we can exploit numpy broadcasting + logaddexp
        #so first we build a vector of scores S = [(w^T x1 + b). . .(w^T xn + b)]
        #then we reshape S to a 1-D array of shape (n_samples, 1)
        S = (vcol(w).T @ DTR).ravel() + b

        #then we exploit broadcasting to compute -z_i * (w^T x_i + b) -> in code it's -ZTR * S -> this term is te full exponent
        exponent = -ZTR * S

        #then we exploit logaddexp: since the log (1 + exp(-z_i * (w^T x_i + b))) can lead to numerical issues, use logaddexp
        #logaddexp(a, b) = log(exp(a) + exp(b))
        #logaddexp(0, exponent) = log(1 + exp(exponent)) -> this is all the second term of the objective function, we just need to compute the mean of it
        logTerm = np.logaddexp(0, exponent)
        avgLogTerms = logTerm.mean()

        return normPenalty + avgLogTerms
    
    #so, the outer function has to invoke the scipy optimizer (fmin_l_bfgs_b) passing the function logreg_obj and the initial x0 which is a numpy array of all zeros and approx_grad = True
    #in this version, we will not manually compute the gradient of the function, but we will use the approx_grad = True option of fmin_l_bfgs_b
    #this will be slower, but we will not have to compute the gradient manually
    xf = opt.fmin_l_bfgs_b(func = logreg_obj, x0 = np.zeros(DTR.shape[0]+1), approx_grad=True)
    
    #xf is a tuple with the first element being the minimum point of the function f, the second element being the value of the function f at the minimum point xf[0], and the third element being a dictionary with information about the optimization process
    
    #extract w_min, b_min
    w_min = xf[0][:-1] #weights which minimize the objective function
    b_min = xf[0][-1]  #bias which minimizes the objective function

    #extract value of objective function in (w_min, b_min)
    objMin = xf[1] 
    
    return w_min, b_min, objMin


In [50]:
#retrieve the parameters which minimize the objective function
#change lambda from 10^(-3) to 1.0, by incrementing it by a decimal order every time
parameters_l = {} #key: lambda, value: (w_min, b_min)
lambdas = [10**(-3), 10**(-2), 10**(-1), 1.0]
for l in lambdas:
    w_min, b_min, objMin = trainLogReg(DTR, LTR, l)
    parameters_l[l] = (w_min, b_min)
    print(f"lambda: {l}, w_min: {w_min}, b_min: {b_min}")
    print(f"Value of the objective function at the minimum point: {objMin}")

lambda: 0.001, w_min: [ 1.72973348  0.98291151 -4.54960055 -7.12479414], b_min: 20.952610898785906
Value of the objective function at the minimum point: 0.11000090355257285
lambda: 0.01, w_min: [ 0.55673154  0.35523499 -2.78054068 -2.76755649], b_min: 13.963100856019771
Value of the objective function at the minimum point: 0.24296227288058903
lambda: 0.1, w_min: [-0.19685817 -0.0212446  -1.10336574 -0.80719181], b_min: 8.175517115004444
Value of the objective function at the minimum point: 0.4539406895961112
lambda: 1.0, w_min: [-0.11040223 -0.02898697 -0.2478712  -0.14950474], b_min: 2.3109467688182845
Value of the objective function at the minimum point: 0.6316436205357172


Now, we've compute the model parameters $(\mathbf{w}, b)$. <br>
So, we just jave to compute the posterior log-probability, by computing for each **validation** (**not training!**) sample $\mathbf{x}_t$:
$$
s(\mathbf{x}_t) = \mathbf{w}^T \mathbf{x}_t + b
$$
using $(\mathbf{w}, b)$ found before.

In [53]:
#compute log posteriors ratios using DVAL samples
scores_l = {} #key: lambda, value: (log posterior ratio for each sample in DVAL)

for l in lambdas:
    p = parameters_l[l]
    w, b = p[0], p[1]
    S = (vcol(w).T @ DVAL).ravel() + b
    scores_l[l] = S
    print(f"lambda: {l}, log posterior ratio: {S}")



lambda: 0.001, log posterior ratio: [ 7.34573105 -1.39291983  7.8487674  -3.59511308  7.51568146 10.3192656
  5.64864932 -2.7628981  -5.93601189  0.05822314 -0.35935684 -7.9592753
 -7.70820105 -0.6649525   5.32463218  4.8301535  -5.01446268  5.29198042
  5.46278923 -5.33168744 -4.21320866  8.17022076 -0.38296579 -6.28666092
 -2.77750163  3.5185371   6.79550277  0.71193003  4.88293213 -1.97790102
 -8.15024837 -6.56525144  5.70188711  4.92242477]
lambda: 0.01, log posterior ratio: [ 3.98610179 -1.16283271  4.50320903 -1.92101139  4.08046413  5.5606327
  3.13308834 -1.34636372 -2.75766427  0.0309689   0.05637192 -3.90249551
 -4.9201439  -0.18138284  2.59427587  2.45534757 -3.6194004   2.89055777
  3.19322561 -2.4393109  -2.52744802  4.19528783  0.04099808 -3.2731614
 -1.64716429  1.57455467  3.63383503  0.13307529  3.09855156 -0.80246374
 -5.11842021 -2.63609133  2.89086948  2.76383762]
lambda: 0.1, log posterior ratio: [ 1.70462274 -0.38947037  1.99003599 -0.90194033  1.82896247  2.32921

Then, we know that $s(\mathbf{x}_t) = \mathbf{w}^T \mathbf{x}_t + b = 0$ is the **decision surface** which separates the two classes. So, we can perform nclass assignments by thresholding the scores with 0 (i.e. $S[i] > 0 \implies LP[i] = 1$, where $LP$ is the array of predicted)

In [54]:
#Perform class assignments
LP_l = {} #key: lambda, value: (log posterior ratio for each sample in DVAL)
for l in lambdas:
    score = scores_l[l]
    LP = np.zeros(score.shape) #predicted labels array
    LP[score > 0] = 1 #assign label 1 to samples with score > 0
    LP[score < 0] = 0 #assign label 0 to samples with score < 0
    LP_l[l] = LP #store the predicted labels for each lambda

#compute error rates 
err_l = {} #key: lambda, value: error rate
for l in lambdas:
    LP = LP_l[l]
    err = (LP != LVAL).sum() / float(LVAL.size) * 100
    err_l[l] = err #store the error rate for each lambda
    print(f"Error rate for lambda {l}: {err}")

Error rate for lambda 0.001: 8.823529411764707
Error rate for lambda 0.01: 8.823529411764707
Error rate for lambda 0.1: 11.76470588235294
Error rate for lambda 1.0: 14.705882352941178
